In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/water-potability/water_potability.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Potability'].value_counts()/len(df['Potability'])

In [ ]:
df.isnull().sum()

# Univariate Analysis

In [ ]:
df.columns

In [ ]:
list1 = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity']

In [ ]:
fig = plt.figure(figsize = (14,40))
a = 10 #rows
b = 2 #columns
c = 1 #increment

for i in list1:
    
    plt.subplot(a,b,c)
  
    sns.distplot(df[i], color='firebrick')
    
    
    c= c+1
    
    
    
plt.show()

In [ ]:
fig = plt.figure(figsize = (14,40))
a = 10 #rows
b = 2 #columns
c = 1 #increment

for i in list1:
    plt.subplot(a,b,c)
    plt.xlabel(i, fontsize=15)
    sns.boxplot(x=df[i], color='firebrick')
    c= c+1
    
plt.show()

In [ ]:
sns.pairplot(df, hue='Potability')

In [ ]:
df.isnull().any(axis=1).sum()

In [ ]:
df['ph'].fillna(df['ph'].mean(), inplace=True)

In [ ]:
df['ph'].isnull().sum()

In [ ]:
df['Sulfate'].fillna(df['Sulfate'].mean(), inplace=True)

In [ ]:
df['Sulfate'].isnull().sum()

In [ ]:
df['Trihalomethanes'].fillna(df['Trihalomethanes'].mean(), inplace=True)

In [ ]:
df['Trihalomethanes'].isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
X=df[df.columns[:-1]]

In [ ]:
X.head()

In [ ]:
y = df['Potability']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=0)

In [ ]:
scaled = StandardScaler()

In [ ]:
X_train_scaled = scaled.fit_transform(X_train)

In [ ]:
X_test_scaled = scaled.transform(X_test)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, classification_report, confusion_matrix

In [ ]:
Log_reg = LogisticRegression(solver='liblinear', penalty='l2',class_weight='balanced')

In [ ]:
Log_reg.fit(X_train_scaled, y_train)

In [ ]:
y_pred= Log_reg.predict(X_test_scaled)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=3,  min_samples_leaf=30)

In [ ]:
dt.fit(X_train_scaled, y_train)

In [ ]:
y_pred1=dt.predict(X_test_scaled)

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
print(confusion_matrix(y_test, y_pred1))

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Random Forest

In [ ]:
rf = RandomForestClassifier()

In [ ]:
param_grid = {'n_estimators':[300,500], 'criterion':['gini', 'entropy'],'max_depth':[3,5], 'min_samples_leaf':[30,50],'max_features':[5,9]}

In [ ]:
gscv_rf = GridSearchCV(rf,param_grid=param_grid, scoring='roc_auc', n_jobs=1, verbose=3)

In [ ]:
gscv_rf.fit(X_train_scaled, y_train)

In [ ]:
print(gscv_rf.best_params_)

In [ ]:
print(gscv_rf.best_score_)
print(gscv_rf.best_estimator_)
print(gscv_rf.scorer_)

In [ ]:
y_pred_rf = gscv_rf.predict(X_test_scaled)

In [ ]:
print(roc_auc_score(y_true=y_test,y_score=gscv_rf.predict_proba(X_test_scaled)[:, 1]))
print(confusion_matrix(y_test,y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# AdaBoostClassifier

In [ ]:
ada =  AdaBoostClassifier()

In [ ]:
param_grid = {'n_estimators':[500,700],'learning_rate':[0.01,0.025, 0.05]}

In [ ]:
gscv_ada = GridSearchCV(ada,param_grid=param_grid, cv=3, scoring='roc_auc', verbose=3, n_jobs=1)

In [ ]:
gscv_ada.fit(X_train_scaled,y_train)

In [ ]:
print(gscv_ada.best_params_)

In [ ]:
print(gscv_ada.best_score_)

In [ ]:
y_pred_ada = gscv_ada.predict(X_test_scaled)

In [ ]:

print(roc_auc_score(y_true=y_test,y_score=gscv_ada.predict_proba(X_test_scaled)[:, 1]))
print(confusion_matrix(y_test,y_pred_ada))
print(classification_report(y_test,y_pred_ada))

# Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier()

In [ ]:
param_grid = {'n_estimators':[300,500],
              'learning_rate':[0.01,0.001],
              'max_depth': [3,5],
             'min_samples_leaf':[30,50]}

In [ ]:
gscv_gb = GridSearchCV(gb, param_grid=param_grid, scoring='roc_auc', verbose=3, n_jobs=-1,cv=3)

In [ ]:
gscv_gb.fit(X_train_scaled, y_train)

In [ ]:
print(gscv_gb.best_params_)
print(gscv_gb.best_score_)

In [ ]:
y_pred_gb = gscv_gb.predict(X_test_scaled)

In [ ]:
print(roc_auc_score(y_true=y_test, y_score=gscv_gb.predict_proba(X_test_scaled)[:,1]))
print(classification_report(y_test,y_pred_gb))
print(confusion_matrix(y_test,y_pred_gb))

# XG Boosting

In [ ]:
xg = XGBClassifier()

In [ ]:

model_parameters = { 'n_estimators': [300,500],
                    'learning_rate': [0.01, 0.025, 0.001],
                    'max_depth': [2, 3],
                    'min_child_weight': [ 20, 30]
                    }

In [ ]:
gscv_xg = GridSearchCV(xg, param_grid=model_parameters, cv=3, verbose=3, n_jobs=1, scoring='roc_auc')

In [ ]:
gscv_xg.fit(X_train_scaled, y_train)

In [ ]:
gscv_xg.best_params_

In [ ]:
gscv_xg.best_score_

In [ ]:
y_pred_xg = gscv_xg.predict(X_test_scaled)

In [ ]:
print(roc_auc_score(y_true=y_test, y_score=gscv_xg.predict_proba(X_test_scaled)[:,1]))
print(classification_report(y_test,y_pred_xg))
print(confusion_matrix(y_test,y_pred_xg))

In [ ]:
def plot_roc_curve( y_pred_rf, y_pred_ada, y_pred_gb, y_pred_xg , *args):
    '''
    Generate test roc curve
    '''

    AUC_rf = roc_auc_score(y_test, y_pred_rf)
    AUC_ad = roc_auc_score(y_test, y_pred_ada)
    AUC_grd = roc_auc_score(y_test, y_pred_gb)
    AUC_xg = roc_auc_score(y_test, y_pred_xg)
    if len(args) == 0:
        print("RF_AUC =" ,AUC_rf )
        print("ADA_AUC = ", AUC_ad)
        print("GB_AUC=" , AUC_grd )
        print("XG_AUC=", AUC_xg )
        fpr_rf, tpr_rf, thresholds = roc_curve(y_test, y_pred_rf)
        fpr_ada, tpr_ada, thresholds = roc_curve(y_test, y_pred_ada)
        fpr_gb, tpr_gb, thresholds = roc_curve(y_test, y_pred_gb)
        fpr_xg, tpr_xg, thresholds = roc_curve(y_test, y_pred_xg)
        roc_plot(fpr_rf, tpr_rf, fpr_ada, tpr_ada, fpr_gb, tpr_gb, fpr_xg, tpr_xg)

    else:
        AUC_Valid = roc_auc_score(args[0], args[1])
        print("Train AUC = ", AUC_Train)
        print("Test AUC = ", AUC_Test)
        print("Validation AUC = ", AUC_Valid)
        fpr, tpr, thresholds = roc_curve(y_train, train_pred_prob)
        fpr_tst, tpr_tst, thresholds = roc_curve(y_test, test_pred_prob)
        fpr_val, tpr_val, thresholds = roc_curve(args[0], args[1])
        roc_plot(fpr, tpr, fpr_tst, tpr_tst, fpr_val, tpr_val)

In [ ]:
def roc_plot(fpr_rf, tpr_rf, fpr_ad, tpr_ad, fpr_grd, tpr_grd, fpr_xg, tpr_xg, *args):
    '''
    Generates roc plot
    '''

    fig = plt.plot(fpr_rf, tpr_rf, label='ROC_RF')
    fig = plt.plot(fpr_ad, tpr_ad, label='ROC_AD')
    fig = plt.plot(fpr_grd, tpr_grd, label= 'ROC_GRD')
    fig = plt.plot(fpr_xg, tpr_xg, label = 'ROC_XG')

    if len(args) == 0:
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.title("ROC curve using ")
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

    else:
        fig = plt.plot(args[0], args[1], label='Validation')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.title("ROC curve using ")
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

In [ ]:
plt.subplots(figsize=(15, 10))
RF = gscv_rf.predict_proba(X_test_scaled)[:, 1]
Ada = gscv_ada.predict_proba(X_test_scaled)[:, 1]
GRD = gscv_gb.predict_proba(X_test_scaled)[:, 1]
XG = gscv_xg.predict_proba(X_test_scaled)[:, 1]


plot_roc_curve(RF, Ada,
               GRD, XG)